In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

In [2]:
train  = pd.read_csv('competition_data/train.csv')
test  = pd.read_csv('competition_data/test.csv')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 100)

In [3]:
train

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,country,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,1.0,5.0,1.0,5.0,1.0,1.0,USA,3,553,6,4.0,3.0,5.0,1.0,3.0,5.0,5.0,3.0,5.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,2.0,1,3.0,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,1.0,4.0,5.0,3.0,1.0,2.0,4.0,5.0,1.0,3.0,1.0,1.0,5.0,3.0,2.0,5.0,USA,5,85,120,4.0,2.0,3.0,5.0,3.0,2.0,5.0,1.0,2.0,2.0,1,1,1,1,1,0,1,0,0,1,0,0,1,1,1,1,4.0,2,2.0,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,2.0,5.0,5.0,5.0,1.0,3.0,5.0,3.0,5.0,2.0,2.0,1.0,2.0,4.0,2.0,5.0,NLD,9,108,100,1.0,2.0,3.0,1.0,5.0,5.0,3.0,4.0,5.0,2.0,1,1,0,1,1,0,1,1,0,1,0,0,1,1,1,1,2.0,1,1.0,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,5.0,2.0,2.0,4.0,4.0,2.0,4.0,5.0,4.0,3.0,3.0,4.0,3.0,4.0,4.0,2.0,USA,2,121,139,3.0,3.0,3.0,4.0,5.0,3.0,4.0,4.0,3.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,1.0,3,1.0,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,4.0,4.0,3.0,5.0,5.0,2.0,4.0,1.0,4.0,2.0,4.0,2.0,3.0,4.0,4.0,4.0,ITA,3,640,216,3.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,2.0,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,1,1.0,2,2.0,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,1.0,4.0,4.0,3.0,4.0,4.0,2.0,5.0,2.0,4.0,1.0,2.0,5.0,4.0,2.0,4.0,USA,12,103,161,2.0,2.0,4.0,3.0,3.0,5.0,3.0,3.0,3.0,3.0,1,1,1,1,1,0,0,0,0,1,0,1,1,1,1,1,2.0,2,2.0,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,1.0,4.0,4.0,4.0,2.0,5.0,5.0,4.0,1.0,5.0,3.0,4.0,4.0,5.0,4.0,5.0,USA,31,106,179,3.0,2.0,4.0,5.0,4.0,3.0,4.0,1.0,2.0,2.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,4.0,1,2.0,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0,5.0,3.0,4.0,USA,17,103,168,1.0,3.0,2.0,5.0,1.0,5.0,3.0,3.0,1.0,1.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,2.0,2,2.0,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1
14998,14998,5.0,5.0,4.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,3.0,5.0,4.0,4.0,1.0,5.0,4.0,5.0,5.0,2.0,5.0,3.0,5.0,3.0,3.0,5.0,USA,14,68,109,1.0,1.0,3.0,5.0,4.0,5.0,5.0,4.0,2.0,1.0,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,3.0,2,2.0,1.0,29,1.0,12.0,4.0,2.0,2.0,2.0,1.0,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 70 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         15000 non-null  int64  
 1   Q1            14959 non-null  float64
 2   Q2            14931 non-null  float64
 3   Q3            14950 non-null  float64
 4   Q4            14929 non-null  float64
 5   Q5            14962 non-null  float64
 6   Q6            14952 non-null  float64
 7   Q7            14924 non-null  float64
 8   Q8            14952 non-null  float64
 9   Q9            14944 non-null  float64
 10  Q10           14928 non-null  float64
 11  Q11           14941 non-null  float64
 12  Q12           14933 non-null  float64
 13  Q13           14960 non-null  float64
 14  Q14           14964 non-null  float64
 15  Q15           14955 non-null  float64
 16  Q16           14967 non-null  float64
 17  Q17           14963 non-null  float64
 18  Q18           14937 non-nu

In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35452 entries, 0 to 35451
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         35452 non-null  int64  
 1   Q1            35333 non-null  float64
 2   Q2            35305 non-null  float64
 3   Q3            35322 non-null  float64
 4   Q4            35327 non-null  float64
 5   Q5            35362 non-null  float64
 6   Q6            35320 non-null  float64
 7   Q7            35310 non-null  float64
 8   Q8            35344 non-null  float64
 9   Q9            35348 non-null  float64
 10  Q10           35232 non-null  float64
 11  Q11           35333 non-null  float64
 12  Q12           35303 non-null  float64
 13  Q13           35356 non-null  float64
 14  Q14           35350 non-null  float64
 15  Q15           35345 non-null  float64
 16  Q16           35367 non-null  float64
 17  Q17           35373 non-null  float64
 18  Q18           35305 non-nu

In [6]:
train = train.drop(['index', 'country'],axis = 1)
test =test.drop(['index', 'country'],axis = 1)
train_y = train['nerdiness']

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 68 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            14959 non-null  float64
 1   Q2            14931 non-null  float64
 2   Q3            14950 non-null  float64
 3   Q4            14929 non-null  float64
 4   Q5            14962 non-null  float64
 5   Q6            14952 non-null  float64
 6   Q7            14924 non-null  float64
 7   Q8            14952 non-null  float64
 8   Q9            14944 non-null  float64
 9   Q10           14928 non-null  float64
 10  Q11           14941 non-null  float64
 11  Q12           14933 non-null  float64
 12  Q13           14960 non-null  float64
 13  Q14           14964 non-null  float64
 14  Q15           14955 non-null  float64
 15  Q16           14967 non-null  float64
 16  Q17           14963 non-null  float64
 17  Q18           14937 non-null  float64
 18  Q19           14947 non-nu

In [8]:
train.isna().sum()

Q1               41
Q2               69
Q3               50
Q4               71
Q5               38
Q6               48
Q7               76
Q8               48
Q9               56
Q10              72
Q11              59
Q12              67
Q13              40
Q14              36
Q15              45
Q16              33
Q17              37
Q18              63
Q19              53
Q20              45
Q21              39
Q22              38
Q23              50
Q24              61
Q25              44
Q26              68
introelapse       0
testelapse        0
surveyelapse      0
TIPI1            53
TIPI2            66
TIPI3            79
TIPI4            64
TIPI5            70
TIPI6            62
TIPI7            64
TIPI8            65
TIPI9            64
TIPI10           80
VCL1              0
VCL2              0
VCL3              0
VCL4              0
VCL5              0
VCL6              0
VCL7              0
VCL8              0
VCL9              0
VCL10             0
VCL11             0


In [9]:
#train을 target과 feature로 나눠줍니다.
#train_ = train.dropna(axis=0)
train_= IterativeImputer().fit_transform(train.iloc[:, :-1])
train_= pd.DataFrame(train_)
test= IterativeImputer().fit_transform(test)
test= pd.DataFrame(test)
#train_ = train

In [10]:
train_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 67 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       15000 non-null  float64
 1   1       15000 non-null  float64
 2   2       15000 non-null  float64
 3   3       15000 non-null  float64
 4   4       15000 non-null  float64
 5   5       15000 non-null  float64
 6   6       15000 non-null  float64
 7   7       15000 non-null  float64
 8   8       15000 non-null  float64
 9   9       15000 non-null  float64
 10  10      15000 non-null  float64
 11  11      15000 non-null  float64
 12  12      15000 non-null  float64
 13  13      15000 non-null  float64
 14  14      15000 non-null  float64
 15  15      15000 non-null  float64
 16  16      15000 non-null  float64
 17  17      15000 non-null  float64
 18  18      15000 non-null  float64
 19  19      15000 non-null  float64
 20  20      15000 non-null  float64
 21  21      15000 non-null  float64
 22

In [11]:
norm=Normalizer()
sc=StandardScaler()

train_S = sc.fit_transform(train_)
train_S = pd.DataFrame(train_S)
test= sc.transform(test) 
test = pd.DataFrame(test)
pca = PCA(n_components=50)
pca.fit(train_S)
#pca.fit(test)
train_pca = pca.transform(train_S)
test_pca = pca.transform(test)

In [12]:
train_pca.shape

(15000, 50)

In [13]:
test_pca.shape

(35452, 50)

In [14]:
pca_columns=['pca_component_1','pca_component_2','pca_component_3',
            'pca_component_4','pca_component_5','pca_component_6',
            'pca_component_7','pca_component_8','pca_component_9',
            'pca_component_10','pca_component_11','pca_component_12',
            'pca_component_13','pca_component_14','pca_component_15',
            'pca_component_16','pca_component_17','pca_component_18',
            'pca_component_19','pca_component_20','pca_component_21',
            'pca_component_22','pca_component_23','pca_component_24',
            'pca_component_25','pca_component_26','pca_component_27',
            'pca_component_28','pca_component_29','pca_component_30',
            'pca_component_31','pca_component_32','pca_component_33',
            'pca_component_34','pca_component_35','pca_component_36',
            'pca_component_37','pca_component_38','pca_component_39',
            'pca_component_40','pca_component_41','pca_component_42',
            'pca_component_43','pca_component_44','pca_component_45',
            'pca_component_46','pca_component_47','pca_component_48',
            'pca_component_49','pca_component_50']
trainDF_pca = pd.DataFrame(train_pca, columns=pca_columns)

In [15]:
pca_columns=['pca_component_1','pca_component_2','pca_component_3',
            'pca_component_4','pca_component_5','pca_component_6',
            'pca_component_7','pca_component_8','pca_component_9',
            'pca_component_10','pca_component_11','pca_component_12',
            'pca_component_13','pca_component_14','pca_component_15',
            'pca_component_16','pca_component_17','pca_component_18',
            'pca_component_19','pca_component_20','pca_component_21',
            'pca_component_22','pca_component_23','pca_component_24',
            'pca_component_25','pca_component_26','pca_component_27',
            'pca_component_28','pca_component_29','pca_component_30',
            'pca_component_31','pca_component_32','pca_component_33',
            'pca_component_34','pca_component_35','pca_component_36',
            'pca_component_37','pca_component_38','pca_component_39',
            'pca_component_40','pca_component_41','pca_component_42',
            'pca_component_43','pca_component_44','pca_component_45',
            'pca_component_46','pca_component_47','pca_component_48',
            'pca_component_49','pca_component_50']
testDF_pca = pd.DataFrame(test_pca, columns=pca_columns)

In [16]:
trainDF_pca

,pca_component_1,pca_component_2,pca_component_3,pca_component_4,pca_component_5,pca_component_6,pca_component_7,pca_component_8,pca_component_9,pca_component_10,pca_component_11,pca_component_12,pca_component_13,pca_component_14,pca_component_15,pca_component_16,pca_component_17,pca_component_18,pca_component_19,pca_component_20,pca_component_21,pca_component_22,pca_component_23,pca_component_24,pca_component_25,pca_component_26,pca_component_27,pca_component_28,pca_component_29,pca_component_30,pca_component_31,pca_component_32,pca_component_33,pca_component_34,pca_component_35,pca_component_36,pca_component_37,pca_component_38,pca_component_39,pca_component_40,pca_component_41,pca_component_42,pca_component_43,pca_component_44,pca_component_45,pca_component_46,pca_component_47,pca_component_48,pca_component_49,pca_component_50
0,1.254186,1.842028,-2.043757,-0.335779,1.314864,-2.416796,-4.875066,-0.009773,0.365336,-0.418379,-0.316461,0.069443,1.586605,1.984663,0.912951,0.974330,-0.694352,-0.625445,0.157670,-2.365661,-1.004486,-0.449968,0.153648,0.350127,-1.392956,-0.719641,-1.455349,2.594039,-0.757356,0.338053,-0.002859,-0.530968,0.998564,0.235825,0.677662,0.429332,0.246350,2.786480,-0.202173,-0.310429,-1.548929,2.068067,-0.459945,-0.123308,0.230834,-0.053185,-0.846848,0.635877,-0.227453,-0.656779
1,1.035007,-2.580645,0.215523,2.878089,-0.604978,0.152375,0.612644,-2.235052,-1.768235,-1.768444,-1.576345,0.464056,-0.083873,0.069065,-0.272438,0.220795,0.281275,-0.656123,0.156059,0.269523,-0.605972,0.131752,0.572833,-0.586348,0.862941,1.339692,-0.769483,-0.906271,-0.125261,-0.167419,0.194152,-0.024666,-0.860128,-0.381135,-1.479497,-0.371277,1.008753,-0.954681,0.891064,0.311667,0.662833,0.982621,-0.651134,-0.399023,0.464551,-0.376599,0.922171,-0.092557,0.276501,0.721987
2,-0.835003,-1.022806,0.412446,-0.337922,3.072088,1.087967,-0.226709,-0.824443,0.318791,1.991815,0.890304,-1.461048,1.108011,1.482081,-1.047950,1.615928,-0.025974,-1.235921,-0.753025,-0.089484,-0.233015,-0.007596,-0.419891,0.801635,0.212182,1.280432,-0.678806,-0.000105,0.160722,0.783408,1.393649,-0.414546,-0.564971,1.275320,-1.189002,-0.710988,-1.276675,0.958407,-0.470601,0.098366,1.227178,0.021168,-0.119396,-1.319884,1.083927,-0.518423,-0.163821,0.662767,0.228275,2.017556
3,1.726889,0.283176,-1.716530,-1.811432,-0.533763,-0.093356,-0.532687,0.372651,-0.102931,-0.059669,1.253375,-0.141132,-0.621669,-0.222089,-0.494399,-1.030036,0.279835,1.120606,-0.816024,0.282162,-1.306361,1.762471,0.317693,-0.847976,-0.300206,0.579325,-0.139276,0.488501,0.811923,-0.553237,0.200422,-0.444366,-0.162940,-0.432417,1.423788,1.570240,-0.278706,-0.690598,-1.464712,0.348324,0.136292,-0.435010,0.652070,0.812964,1.093182,-0.474894,0.096261,0.646327,0.877077,0.366370
4,1.960015,1.878120,0.437608,-1.583157,-0.613325,-0.552539,-0.021125,-0.125607,1.574766,0.493540,-0.614073,0.107864,0.744965,0.683016,-0.308630,-0.305154,-0.207414,0.436124,-0.534220,-0.824942,0.063246,1.495375,-0.456492,0.170827,-0.292971,-1.315109,-0.860221,2.223705,-1.536606,0.478837,-0.957260,-0.193083,0.185973,0.775242,0.049992,-0.091707,-1.380233,0.072556,-1.107310,-0.977705,0.575116,-0.143131,0.073389,0.500040,-0.341267,2.399208,-0.473621,0.129683,-0.138551,0.794895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.687817,-0.649214,-1.259046,1.572435,0.285266,-0.531820,-1.660681,0.852526,-1.244036,-1.815040,0.812395,-1.857110,-0.107439,-0.253175,0.638610,0.298325,-0.156360,0.009792,0.369399,0.063496,0.721211,0.262583,-0.184470,0.297042,0.567631,-0.518519,0.716294,-0.594443,0.092178,-0.990785,0.284854,1.128727,-0.342612,0.257357,-0.010743,-0.192792,-1.119517,0.137975,0.260657,0.553565,-0.119558,-0.120937,-0.906852,-1.081120,-0.210751,-0.259217,-0.206439,-0.377030,-0.357818,-0.580845
14996,-1.258090,0.077655,0.572712,1.974651,1.211733,-2.264104,1.626131,-1.946979

In [17]:
testDF_pca

,pca_component_1,pca_component_2,pca_component_3,pca_component_4,pca_component_5,pca_component_6,pca_component_7,pca_component_8,pca_component_9,pca_component_10,pca_component_11,pca_component_12,pca_component_13,pca_component_14,pca_component_15,pca_component_16,pca_component_17,pca_component_18,pca_component_19,pca_component_20,pca_component_21,pca_component_22,pca_component_23,pca_component_24,pca_component_25,pca_component_26,pca_component_27,pca_component_28,pca_component_29,pca_component_30,pca_component_31,pca_component_32,pca_component_33,pca_component_34,pca_component_35,pca_component_36,pca_component_37,pca_component_38,pca_component_39,pca_component_40,pca_component_41,pca_component_42,pca_component_43,pca_component_44,pca_component_45,pca_component_46,pca_component_47,pca_component_48,pca_component_49,pca_component_50
0,-3.109030,-0.039670,0.369129,0.873678,-2.640105,1.299457,-1.141292,2.494498,3.011604,-0.419226,-0.822542,0.141324,-0.416678,-0.783218,0.396466,0.337265,-0.095121,0.039168,0.487205,0.415356,1.340794,-1.201575,0.119752,-1.215669,0.272534,0.210737,0.728506,-1.259352,-1.087807,0.275099,-1.094690,-1.486209,-0.236751,-0.332107,-0.178077,1.013656,-0.364427,-0.346756,-0.273966,1.328698,1.188384,0.766931,-1.201024,0.924976,0.240579,0.293460,0.008922,0.455136,-0.105328,1.193816
1,-2.664951,2.171836,0.261763,-1.091075,-0.024331,0.987031,0.187205,-0.578727,-0.528255,0.072706,0.825935,-0.434790,-0.889415,-0.973465,1.468082,0.875144,0.341923,1.029910,-0.506816,0.044290,1.189411,-0.106174,-1.586883,0.681476,0.115163,0.013262,-0.711111,-0.127117,-0.721583,0.799944,0.257054,0.141210,-1.034454,-0.727493,0.137214,-1.641402,-0.506498,1.386468,-0.253685,1.573169,1.300377,-0.104245,0.171276,1.167304,1.495113,-0.837097,-0.738022,-1.902245,-0.422614,-0.921144
2,-2.042802,1.367620,-0.327404,0.658323,-2.018000,-3.445680,1.222697,-0.425990,0.687119,0.219866,-0.025832,-1.201448,0.022808,-0.194321,0.846158,-0.250382,-0.317668,-0.181211,0.936109,-0.093623,-0.769451,-0.202301,0.303197,0.352728,0.907374,-0.542723,-0.164942,-0.376546,-0.386670,0.707940,-0.611101,-0.753119,0.167837,0.343983,-0.265102,-0.639414,-1.202633,1.241823,-0.025635,-0.379968,1.145917,0.294345,-0.216754,-0.590154,1.090400,-0.798565,-0.100143,0.004737,-0.353281,0.872563
3,-1.434226,-1.241781,0.276316,-1.355189,-0.337275,2.558549,-0.341244,1.739043,-0.581602,0.459060,-0.879979,-1.052701,-0.228439,-0.865118,-0.761324,0.209911,0.543444,-0.124782,0.024508,0.927763,0.444720,0.230500,-0.638515,0.721442,0.090845,0.170796,0.424629,-1.253123,1.194834,1.796703,0.024627,0.976847,-0.128887,-0.265026,0.310802,-0.399957,-0.218737,-0.574367,1.098296,0.288763,-1.286583,1.431148,-1.396866,0.788973,0.161775,-0.732914,1.214105,-0.178425,-0.277395,0.056530
4,0.485773,6.228365,2.356545,0.936582,-0.366590,-1.718887,1.608995,-0.200905,0.954710,-0.776962,-0.206768,-1.200934,0.490717,0.652742,-0.154225,-0.039336,-0.300161,-1.553379,0.875280,-0.332470,-1.265776,-0.773359,0.885927,0.803917,-0.232754,-1.389779,0.976861,-2.152868,-0.109775,0.082098,2.007897,0.640036,-0.605323,0.039175,0.766272,1.366687,0.178636,-0.115416,0.371387,0.397569,0.685717,-0.089636,0.627976,0.176553,-0.838675,-1.238992,-0.697876,-0.448954,1.380295,-0.119210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35447,1.501498,7.260579,5.460047,-0.569602,-1.428408,-1.199342,-1.023018,-0.163449,0.828713,-0.845368,-1.403223,0.798977,-0.012353,-0.070270,1.557570,0.072054,-0.314108,1.225013,-0.673595,-1.848627,-2.279607,1.888365,-0.590394,0.618735,-2.013084,-0.486481,-1.420933,1.598776,1.223374,0.107206,-0.093641,-0.118240,-0.163728,1.026638,0.841596,-1.117311,0.815914,-0.507320,0.544707,0.570790,0.995737,2.138421,0.582151,-0.671992,-0.141071,0.547528,-0.129724,0.485648,2.995457,1.904288
35448,-2.640624,2.467405,-0.455391,-0.092649,-0.959869,-1.133936,-0.241034,0.112378,-1

In [18]:
y_target=train['nerdiness']

In [19]:
start_time=time.time()
rcf = RandomForestClassifier()
random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': [2],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [4, 6, 8],
               'min_samples_split': [5, 7,10],
               'n_estimators': [20]}
clf = RandomizedSearchCV(estimator = rcf, param_distributions = random_search, n_iter = 10, 
                               cv = 4, verbose= 1, random_state= 101, n_jobs = -1)
clf.fit(X=trainDF_pca,y=y_target)
end_time=time.time()
print('소요 시간 : ')
print(end_time-start_time)

Fitting 4 folds for each of 10 candidates, totalling 40 fits
소요 시간 : 
4.555945158004761


In [20]:
'''
start_time=time.time()
lgb= lgb.LGBMClassifier(
            n_estimators=1000
        )
parametes= {
    'num_leaves':[20,40,60,80,100],'min_child_samples':[5,10,15],
    'max_depth':[-1,5,10,20],'learning_rate':[0.05,0.1,0.2],'reg_alpha':[0,0.01,0.03]
}
clf=RandomizedSearchCV(lgb,parametes,scoring='roc_auc',n_iter=100)
clf.fit(X=trainDF_pca,y=y_target)
end_time=time.time()
print('소요 시간 : ')
print(end_time-start_time)
'''

"\nstart_time=time.time()\nlgb= lgb.LGBMClassifier(\n            n_estimators=1000\n        )\nparametes= {\n    'num_leaves':[20,40,60,80,100],'min_child_samples':[5,10,15],\n    'max_depth':[-1,5,10,20],'learning_rate':[0.05,0.1,0.2],'reg_alpha':[0,0.01,0.03]\n}\nclf=RandomizedSearchCV(lgb,parametes,scoring='roc_auc',n_iter=100)\nclf.fit(X=trainDF_pca,y=y_target)\nend_time=time.time()\nprint('소요 시간 : ')\nprint(end_time-start_time)\n"

In [21]:
# 재학습 위한 공간

In [22]:
#lgbm_clf.fit(train_x, train_y)
lgbm_pred = clf.predict(testDF_pca)

In [23]:
submission = pd.read_csv('competition_data/sample_submission.csv')

submission

,index,nerdiness
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1
...,...,...
35447,35447,-1
35448,35448,-1
35449,35449,-1
35450,35450,-1


In [24]:
submission["nerdiness"] = lgbm_pred

In [25]:
submission

,index,nerdiness
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
35447,35447,1
35448,35448,1
35449,35449,1
35450,35450,1


In [26]:
submission.to_csv("baseline_notNA-iterative-withNorm-PCA-RF.csv", index = False)